## Target in SQL

In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# %sql duckdb:///:default: # not working on vscode

%sql duckdb:///:memory:

In [2]:
# importamos datos

dataset_path = 'C:/Users/Cristian Burich/Desktop/MA/segundo/eyf/datasets/' # '/content/drive/MyDrive/DMEyF/2024/datos/'
dataset_file = 'competencia_01_crudo.csv'

In [3]:
%%sql
create or replace table competencia_01_crudo as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}")

,Success


In [4]:
%%sql
create or replace table competencia_01 as
with periodos as (
    select distinct foto_mes from competencia_01_crudo
), clientes as (
    select distinct numero_de_cliente from competencia_01_crudo
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        c.*
        , if(c.numero_de_cliente is null, 0, 1) as mes_0   -- si el numero de cliente existe para el período, mes_0 es 1, else = 0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1  --creo que mantiene el IF, sobre la fila siguiente
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2 
        , if(mes_0 = 1 and mes_1 = 0, 'BAJA+1', if(mes_0 = 1 and mes_2 = 0, 'BAJA+2', 'CONTINUA')) as clase_ternaria
    from todo t
    left join competencia_01_crudo c using (numero_de_cliente, foto_mes)
) select
  * EXCLUDE (mes_0, mes_1, mes_2)
from clase_ternaria
where mes_0 = 1

,Success


In [5]:
%sql select count(*) from competencia_01

,count_star()
0,981946


In [6]:
%sql select count(*) from competencia_01 where clase_ternaria = 'BAJA+1' -- 4628 vs 1206


,count_star()
0,4628


In [7]:
%sql select count(*) from competencia_01 where clase_ternaria = 'BAJA+2' --3972

,count_star()
0,3972


In [8]:
# Guardamos dataset

# %sql COPY competencia_01 TO '{{dataset_path}}competencia_01_sql_v3.csv' (FORMAT CSV, HEADER)

### Feature engineering

In [9]:
campos = ['cpayroll_trx', 'Visa_msaldototal', 'mautoservicio', 'mpasivos_margen', 'ctrx_quarter', 'mcomisiones', 'ccomisiones_mantenimiento', 'mautoservicio', 'mcaja_ahorro', 'mpayroll', 'mcuenta_corriente']


In [10]:
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n, {campo} - lag({campo}, 1) over (partition by numero_de_cliente order by foto_mes) as delta_1_{campo}"
print(nuevos_features)


, cpayroll_trx - lag(cpayroll_trx, 1) over (partition by numero_de_cliente order by foto_mes) as delta_1_cpayroll_trx
, Visa_msaldototal - lag(Visa_msaldototal, 1) over (partition by numero_de_cliente order by foto_mes) as delta_1_Visa_msaldototal
, mautoservicio - lag(mautoservicio, 1) over (partition by numero_de_cliente order by foto_mes) as delta_1_mautoservicio
, mpasivos_margen - lag(mpasivos_margen, 1) over (partition by numero_de_cliente order by foto_mes) as delta_1_mpasivos_margen
, ctrx_quarter - lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as delta_1_ctrx_quarter
, mcomisiones - lag(mcomisiones, 1) over (partition by numero_de_cliente order by foto_mes) as delta_1_mcomisiones
, ccomisiones_mantenimiento - lag(ccomisiones_mantenimiento, 1) over (partition by numero_de_cliente order by foto_mes) as delta_1_ccomisiones_mantenimiento
, mautoservicio - lag(mautoservicio, 1) over (partition by numero_de_cliente order by foto_mes) as delta_1_mautos

In [11]:
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n, {campo} - lag({campo}, 2) over (partition by numero_de_cliente order by foto_mes) as delta_2_{campo}"
print(nuevos_features)


, cpayroll_trx - lag(cpayroll_trx, 2) over (partition by numero_de_cliente order by foto_mes) as delta_2_cpayroll_trx
, Visa_msaldototal - lag(Visa_msaldototal, 2) over (partition by numero_de_cliente order by foto_mes) as delta_2_Visa_msaldototal
, mautoservicio - lag(mautoservicio, 2) over (partition by numero_de_cliente order by foto_mes) as delta_2_mautoservicio
, mpasivos_margen - lag(mpasivos_margen, 2) over (partition by numero_de_cliente order by foto_mes) as delta_2_mpasivos_margen
, ctrx_quarter - lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as delta_2_ctrx_quarter
, mcomisiones - lag(mcomisiones, 2) over (partition by numero_de_cliente order by foto_mes) as delta_2_mcomisiones
, ccomisiones_mantenimiento - lag(ccomisiones_mantenimiento, 2) over (partition by numero_de_cliente order by foto_mes) as delta_2_ccomisiones_mantenimiento
, mautoservicio - lag(mautoservicio, 2) over (partition by numero_de_cliente order by foto_mes) as delta_2_mautos

In [12]:
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n, {campo} - lag({campo}, 3) over (partition by numero_de_cliente order by foto_mes) as delta_3_{campo}"
print(nuevos_features)


, cpayroll_trx - lag(cpayroll_trx, 3) over (partition by numero_de_cliente order by foto_mes) as delta_3_cpayroll_trx
, Visa_msaldototal - lag(Visa_msaldototal, 3) over (partition by numero_de_cliente order by foto_mes) as delta_3_Visa_msaldototal
, mautoservicio - lag(mautoservicio, 3) over (partition by numero_de_cliente order by foto_mes) as delta_3_mautoservicio
, mpasivos_margen - lag(mpasivos_margen, 3) over (partition by numero_de_cliente order by foto_mes) as delta_3_mpasivos_margen
, ctrx_quarter - lag(ctrx_quarter, 3) over (partition by numero_de_cliente order by foto_mes) as delta_3_ctrx_quarter
, mcomisiones - lag(mcomisiones, 3) over (partition by numero_de_cliente order by foto_mes) as delta_3_mcomisiones
, ccomisiones_mantenimiento - lag(ccomisiones_mantenimiento, 3) over (partition by numero_de_cliente order by foto_mes) as delta_3_ccomisiones_mantenimiento
, mautoservicio - lag(mautoservicio, 3) over (partition by numero_de_cliente order by foto_mes) as delta_3_mautos

In [13]:
%%sql
create or replace table competencia_01 as
select *
, cpayroll_trx - lag(cpayroll_trx, 1) over (partition by numero_de_cliente order by foto_mes) as delta_1_cpayroll_trx
, Visa_msaldototal - lag(Visa_msaldototal, 1) over (partition by numero_de_cliente order by foto_mes) as delta_1_Visa_msaldototal
, mautoservicio - lag(mautoservicio, 1) over (partition by numero_de_cliente order by foto_mes) as delta_1_mautoservicio
, mpasivos_margen - lag(mpasivos_margen, 1) over (partition by numero_de_cliente order by foto_mes) as delta_1_mpasivos_margen
, ctrx_quarter - lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as delta_1_ctrx_quarter
, mcomisiones - lag(mcomisiones, 1) over (partition by numero_de_cliente order by foto_mes) as delta_1_mcomisiones
, ccomisiones_mantenimiento - lag(ccomisiones_mantenimiento, 1) over (partition by numero_de_cliente order by foto_mes) as delta_1_ccomisiones_mantenimiento
, mautoservicio - lag(mautoservicio, 1) over (partition by numero_de_cliente order by foto_mes) as delta_1_mautoservicio
, mcaja_ahorro - lag(mcaja_ahorro, 1) over (partition by numero_de_cliente order by foto_mes) as delta_1_mcaja_ahorro
, mpayroll - lag(mpayroll, 1) over (partition by numero_de_cliente order by foto_mes) as delta_1_mpayroll
, mcuenta_corriente - lag(mcuenta_corriente, 1) over (partition by numero_de_cliente order by foto_mes) as delta_1_mcuenta_corriente
, cpayroll_trx - lag(cpayroll_trx, 2) over (partition by numero_de_cliente order by foto_mes) as delta_2_cpayroll_trx
, Visa_msaldototal - lag(Visa_msaldototal, 2) over (partition by numero_de_cliente order by foto_mes) as delta_2_Visa_msaldototal
, mautoservicio - lag(mautoservicio, 2) over (partition by numero_de_cliente order by foto_mes) as delta_2_mautoservicio
, mpasivos_margen - lag(mpasivos_margen, 2) over (partition by numero_de_cliente order by foto_mes) as delta_2_mpasivos_margen
, ctrx_quarter - lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as delta_2_ctrx_quarter
, mcomisiones - lag(mcomisiones, 2) over (partition by numero_de_cliente order by foto_mes) as delta_2_mcomisiones
, ccomisiones_mantenimiento - lag(ccomisiones_mantenimiento, 2) over (partition by numero_de_cliente order by foto_mes) as delta_2_ccomisiones_mantenimiento
, mautoservicio - lag(mautoservicio, 2) over (partition by numero_de_cliente order by foto_mes) as delta_2_mautoservicio
, mcaja_ahorro - lag(mcaja_ahorro, 2) over (partition by numero_de_cliente order by foto_mes) as delta_2_mcaja_ahorro
, mpayroll - lag(mpayroll, 2) over (partition by numero_de_cliente order by foto_mes) as delta_2_mpayroll
, mcuenta_corriente - lag(mcuenta_corriente, 2) over (partition by numero_de_cliente order by foto_mes) as delta_2_mcuenta_corriente
, cpayroll_trx - lag(cpayroll_trx, 3) over (partition by numero_de_cliente order by foto_mes) as delta_3_cpayroll_trx
, Visa_msaldototal - lag(Visa_msaldototal, 3) over (partition by numero_de_cliente order by foto_mes) as delta_3_Visa_msaldototal
, mautoservicio - lag(mautoservicio, 3) over (partition by numero_de_cliente order by foto_mes) as delta_3_mautoservicio
, mpasivos_margen - lag(mpasivos_margen, 3) over (partition by numero_de_cliente order by foto_mes) as delta_3_mpasivos_margen
, ctrx_quarter - lag(ctrx_quarter, 3) over (partition by numero_de_cliente order by foto_mes) as delta_3_ctrx_quarter
, mcomisiones - lag(mcomisiones, 3) over (partition by numero_de_cliente order by foto_mes) as delta_3_mcomisiones
, ccomisiones_mantenimiento - lag(ccomisiones_mantenimiento, 3) over (partition by numero_de_cliente order by foto_mes) as delta_3_ccomisiones_mantenimiento
, mautoservicio - lag(mautoservicio, 3) over (partition by numero_de_cliente order by foto_mes) as delta_3_mautoservicio
, mcaja_ahorro - lag(mcaja_ahorro, 3) over (partition by numero_de_cliente order by foto_mes) as delta_3_mcaja_ahorro
, mpayroll - lag(mpayroll, 3) over (partition by numero_de_cliente order by foto_mes) as delta_3_mpayroll
, mcuenta_corriente - lag(mcuenta_corriente, 3) over (partition by numero_de_cliente order by foto_mes) as delta_3_mcuenta_corriente
from competencia_01

,Success


In [14]:
%sql COPY competencia_01 TO '{{dataset_path}}competencia_01_sql_v4.csv' (FORMAT CSV, HEADER)

,Success


# Seguimos con el dataset en python

In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle

c:\Users\Cristian Burich\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
dataset_path = 'C:/Users/Cristian Burich/Desktop/MA/segundo/eyf/datasets/' # '/content/drive/MyDrive/DMEyF/2024/datos/'
dataset_file = 'competencia_01_sql_v4.csv'

base_path = 'C:/Users/Cristian Burich/Desktop/MA/segundo/eyf/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'

ganancia_acierto = 273000
costo_estimulo = 7000

data = pd.read_csv(dataset_path+dataset_file)

In [17]:
semillas = [165229,165211,165203,165237,165247]

mes_train = 202104
mes_test = 202106

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [18]:
#data['clase_binaria1'] = 0
data['clase_binaria2'] = 0
#data['clase_binaria1'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)
data['clase_binaria2'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1) # la binaria2 incluye a los BAJA+1

# tirar la que no usas

In [19]:
data.drop(columns=['mprestamos_personales', 'cprestamos_personales'], inplace = True)

In [20]:
train_data = data[data['foto_mes'] == mes_train]
test_data = data[data['foto_mes'] == mes_test]

X_train = train_data.drop(['clase_ternaria', 'clase_peso','clase_binaria2'], axis=1)  # , 'clase_binaria1'
#y_train_binaria1 = train_data['clase_binaria1']
y_train_binaria2 = train_data['clase_binaria2']
w_train = train_data['clase_peso']

X_test = test_data.drop(['clase_ternaria', 'clase_peso','clase_binaria2'], axis=1)  # , 'clase_binaria1'
# y_test_binaria1 = test_data['clase_binaria1']
y_test_binaria2 = test_data['clase_binaria2']
y_test_class = test_data['clase_ternaria']
w_test = test_data['clase_peso']

In [21]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')  # podemos intentar otras imputaciones y ver que tal
Xif = imp_mean.fit_transform(X_test)

In [22]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Parámetros del modelos.
params = {
    'objective': 'binary',
    'metric': 'gan_eval',
    'boosting_type': 'gbdt',
    'max_bin': 31,
    'num_leaves': 31,
    'learning_rate': 0.01,
    'feature_fraction': 0.3,
    'bagging_fraction': 0.7,
    'verbose': 0
}

In [23]:
# train_data1 = lgb.Dataset(X_train, label=y_train_binaria1, weight=w_train)
train_data2 = lgb.Dataset(X_train, label=y_train_binaria2, weight=w_train)

# LGBM

In [24]:

def objective(trial):

    num_leaves = trial.suggest_int('num_leaves', 8, 100),
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.3), # mas bajo, más iteraciones necesita
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 1000),
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0),
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0),

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semillas[0],
        'verbose': -1
    }
    train_data = lgb.Dataset(X_train,
                              label=y_train_binaria2, # eligir la clase
                              weight=w_train)
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=100, # modificar, subit y subir... y descomentar la línea inferior
        # early_stopping_rounds= int(50 + 5 / learning_rate),
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5


storage_name = "sqlite:///" + db_path + "optimization_lgbm.db"
study_name = "exp_009_lgbm"

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-10-12 14:01:42,270] A new study created in RDB with name: exp_009_lgbm


In [25]:
study.optimize(objective, n_trials=1000) # subir subir

[I 2024-10-12 14:01:51,547] Trial 0 finished with value: 145383000.0 and parameters: {'num_leaves': 32, 'learning_rate': 0.17284726812497997, 'min_data_in_leaf': 949, 'feature_fraction': 0.1672654025439539, 'bagging_fraction': 0.8411285764182453}. Best is trial 0 with value: 145383000.0.
[I 2024-10-12 14:02:04,959] Trial 1 finished with value: 149856000.0 and parameters: {'num_leaves': 98, 'learning_rate': 0.16128735324444163, 'min_data_in_leaf': 547, 'feature_fraction': 0.48007842384468513, 'bagging_fraction': 0.43407039465080766}. Best is trial 1 with value: 149856000.0.
[I 2024-10-12 14:02:13,519] Trial 2 finished with value: 154126000.0 and parameters: {'num_leaves': 39, 'learning_rate': 0.03322250539797376, 'min_data_in_leaf': 653, 'feature_fraction': 0.8112389202920626, 'bagging_fraction': 0.9537710609225949}. Best is trial 2 with value: 154126000.0.
[I 2024-10-12 14:02:23,276] Trial 3 finished with value: 151844000.0 and parameters: {'num_leaves': 41, 'learning_rate': 0.10634699

In [26]:
best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor model {best_iter}")
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': study.best_trial.params['num_leaves'],
    'learning_rate': study.best_trial.params['learning_rate'],
    'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
    'feature_fraction': study.best_trial.params['feature_fraction'],
    'bagging_fraction': study.best_trial.params['bagging_fraction'],
    'seed': semillas[0],
    'verbose': 0
}

train_data = lgb.Dataset(X_train,
                          label=y_train_binaria2,
                          weight=w_train)

model = lgb.train(params,
                  train_data,
                  num_boost_round=best_iter)

Mejor cantidad de árboles para el mejor model 79


In [27]:
# guardamos modelos

model.save_model(modelos_path + 'lgb_k009.txt')

In [28]:
# levantamos modelo

model = lgb.Booster(model_file=modelos_path + 'lgb_k009.txt')

In [29]:
# predecimos

y_pred_lgm = model.predict(X_test)

In [30]:
def ganancia_prob(y_pred, y_true, prop = 1):
  ganancia = np.where(y_true == 1, ganancia_acierto, 0) - np.where(y_true == 0, costo_estimulo, 0)
  return ganancia[y_pred >= 0.025].sum() / prop

print("Ganancia LGBM:", ganancia_prob(y_pred_lgm, y_test_binaria2))


Ganancia LGBM: -97258000.0


## Entrenamos en Abril

In [31]:
mes_test = 202106
X_futuro = data[data['foto_mes'] == mes_test]
# y_futuro = X_futuro['clase_ternaria'] # tiene valores pero porque armaste el target como el orto
X_futuro = X_futuro.drop(columns=['clase_ternaria', 'clase_peso','clase_binaria2'])
Xif = imp_mean.fit_transform(X_futuro)

In [32]:
y_pred_rf = model.predict(Xif)

### Salida a Kaggle

In [33]:
# GProbabilidades predichas
predicted_prob = model.predict(Xif, raw_score=False, pred_leaf=False, pred_contrib=False)

# Tomamos numero de cliente y proba
df_predictions = pd.DataFrame({
    'numero_de_cliente': X_futuro['numero_de_cliente'],
    'probability': predicted_prob
})

# Ordenamos por proba de mayor a menor
df_predictions = df_predictions.sort_values(by='probability', ascending=False)

# Nos quedamos con los 12k de mayor probabilidad
df_predictions['Predicted'] = 0  # Default to 0
df_predictions.iloc[:12000, df_predictions.columns.get_loc('Predicted')] = 1  # Set top 10,000 to 1



In [36]:
# Formato Kaggle

K101_009 = df_predictions[['numero_de_cliente', 'Predicted']]

In [37]:
# Guardamos

file_path = 'C:/Users/Cristian Burich/Desktop/MA/segundo/eyf/datasets/exp/KA2001/K101_009.csv'
K101_009.to_csv(file_path, index=False)